In [0]:
dbutils.fs.mkdirs("/Volumes/workspace/ecommerce/ecommerce_data/stream_input")
print("Stream input folder created!")

In [0]:
# Load bronze table
events = spark.table("workspace.ecommerce.events_delta")

# Take small sample
sample_df = events.limit(1000)

# Write CSV files DIRECTLY to stream_input folder (important fix)
sample_df.write.mode("overwrite").option("header", "true").csv(
"/Volumes/workspace/ecommerce/ecommerce_data/stream_input"
)
print("Sample files written directly to stream_input folder!")

In [0]:
stream_df = spark.readStream \
    .schema(events.schema) \
    .option("header", "true") \
    .csv("/Volumes/workspace/ecommerce/ecommerce_data/stream_input")

print("Streaming DataFrame created successfully!")

In [0]:
query = stream_df.writeStream \
.format("delta") \
.outputMode("append") \
.trigger(availableNow=True) \
.option(
    "checkpointLocation",
    "/Volumes/workspace/ecommerce/ecommerce_data/checkpoints/stream_chk"
    ) \
    .start("/Volumes/workspace/ecommerce/ecommerce_data/stream_output")

print("Streaming job started ... ")

In [0]:
for q in spark.streams.active:
    print("Stopping stream:", q.id)
    q.stop()

print("All active streams stopped. ")



In [0]:
dbutils.fs.rm(
"/Volumes/workspace/ecommerce/ecommerce_data/checkpoints/stream_chk",
True
)
print("Old checkpoint folder cleared successfully!")